# Quantum State Discrimination


<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## Overview

Quantum state discrimination (QSD) [1-2] is a fundamental question in quantum communication, quantum computation, and quantum cryptography. In this tutorial, we will explain how to discriminate two orthogonal bipartite pure states $\lvert\psi\rangle$ and $\lvert\phi\rangle$, which satisfies $\langle\psi\lvert\phi\rangle=0$, under the constraint of Local Operations and Classical Communication (LOCC). We refer all the theoretical details to the original paper [3].


## QSD Protocol


Firstly, we want to make the problem definition clear. Consider two spatially separated parties $A$ (Alice) and $B$ (Bob) share a given two-qubit system, the system state is $\lvert\varphi\rangle$ previously distributed by another party $C$ (Charlie). Alice and Bob were only notified that $\lvert\varphi\rangle$ is either $\lvert\psi\rangle$ or $\lvert\phi\rangle$ (both are pure states), satisfying $\langle\psi\lvert\phi\rangle=0$. Then, Charlie provides many copies of $\lvert\psi\rangle$ and $\lvert\phi\rangle$ to them, and he asks Alice and Bob to cooperate with each other to figure out which state they are actually sharing.


Solving this problem under our LOCCNet framework is trivial. As always, let's start with the simplest one-round LOCC protocol with a QNN architecture shown in Figure 1. Then, the difficult lies in the design of an appropriate loss function $L$. Since we choose to let both parties to measure their subsystem, there will be four possible measurement results $m_Am_B\in\{00, 01, 10, 11\}$. To distinguish $\lvert\psi\rangle$ and $\lvert\phi\rangle$, we will label the former state with measurement results $m_Am_B\in\{00, 10\}$ and the latter with $m_Am_B\in\{01, 11\}$. This step can be understood as adding labels to the data in supervised learning. With these labels, we can define the loss function as the probability of guessing wrong label,

$$
L = p_{\lvert\psi\rangle\_01}+p_{\lvert\psi\rangle\_11}+p_{\lvert\phi\rangle\_10}+p_{\lvert\phi\rangle\_00}.
\tag{1}
$$

where $p_{\lvert\psi\rangle\_01}$ stands for the probability of measuring 01 when the input state is $\lvert\psi\rangle$. Then we can begin the training stage to minimize the loss function.


<center><img src="figures/discrimination-fig-circuit.png" height="300" width="450"></center>
<div style="text-align:center">Figure 1: Schematic diagram of state discrimination with LOCCNet. </div>


We summarize the workflow below:


1. Alice and Bob share a two-qubit system, which state is either $\lvert\psi\rangle$ or $\lvert\phi\rangle$. 
2. Alice operates a general rotation gate $U_A$ on her qubit.
3. Alice measures her qubit on the computational basis, and the result $m_A\in \{0, 1\}$. Then, she communicates with Bob about the measurement result through a classical channel.
4.  Bob operates different gates on his qubit depending on Alice's measurement result. If, $m_A=0$ Bob acts $U_{B0}$ on his qubit; If $m_A = 1$, then Bob acts $U_{B1}$. Then, Bob measures his qubit and obtain $m_B \in \{0,1\}$. **Note**: Both $U_{B0}$ and $U_{B1}$ are universal single-qubit gate `u3()`.
5. Calculate the loss function $L = p_{\lvert\psi\rangle\_01}+p_{\lvert\psi\rangle\_11}+ p_{\lvert\phi\rangle\_10}+ p_{\lvert\phi\rangle\_00}$, and use gradient-based optimization methods to minimize it.
6. Repeat 1-5 until the loss function converges.
7. Input the pre-shared state $\lvert\varphi\rangle$ to make a decision and compare with Charlie's answer.




## Simulation with Paddle Quantum 

First, import relevant packages.

In [1]:
import numpy as np
from scipy.stats import unitary_group
import paddle.fluid as fluid
from paddle_quantum.locc import LoccNet

Randomly generate two orthogonal pure states $\lvert\psi\rangle$ and $\lvert\phi\rangle$ by Charlie.

In [2]:
def states_orthogonal_random(n, num=2):
    
    # Randomly generate two orthogonal states
    assert num <= 2 ** n, "return too many orthognal states"
    U = unitary_group.rvs(2 ** n)
    return_list = []
    for i in range(num):
        return_list.append(U[i])
    return return_list

Below is the main part of our LOCC protocol:

In [3]:
class Net (LoccNet):
    def __init__(self):
        super(Net, self).__init__()
        
        # Add the first party Alice 
        # The first parameter 1 stands for how many qubits A holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(1, party_name='Alice')
        
        # Add the first party Bob 
        # The first parameter 1 stands for how many qubits B holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(1, party_name='Bob')
        
        # Initialize Alice's parameter
        self.theta1 = self.create_parameter(shape=[3], attr=fluid.initializer.Uniform(low=0.0, high=2 * np.pi, seed=SEED),
                                            dtype="float64")
        # Initialize Bob's parameter
        # Bob has to prepare two circuits according Alice's measurement result 
        self.theta2 = self.create_parameter(shape=[3], attr=fluid.initializer.Uniform(low=0.0, high=2 * np.pi, seed=SEED),
                                            dtype="float64")
        self.theta3 = self.create_parameter(shape=[3], attr=fluid.initializer.Uniform(low=0.0, high=2 * np.pi, seed=SEED),
                                            dtype="float64")
        
        # Rewrite the input states into density martices
        _states = states_orthogonal_random(2)
        _states = [fluid.dygraph.to_variable(np.outer(init_state, init_state.conjugate())) for init_state in _states]
        
        # Initialize the system by distributing states
        self.set_init_status(_states[0], [('Alice', 0), ('Bob', 0)])
        self.psi = self.init_status
        self.phi = self.reset_state(self.init_status, _states[1], [('Alice', 0), ('Bob', 0)])

    def A_circuit(self, theta, state, res):
        # Alice's local operations
        cir = self.create_ansatz('Alice')
        # Add single-qubit universal gate
        cir.u3(*theta, 0)
        # Run circuit
        after_state = cir.run(state)
        # Measure the circuit and record the measurement results 
        after_state = self.measure(status=after_state, which_qubits=('Alice', 0), results_desired=res)
        return after_state

    def B_circuit(self, theta, state, res):
        # Bob's local operations
        cir = self.create_ansatz('Bob')
        # Add single-qubit universal gate
        cir.u3(*theta, 0)
        # Run circuit
        after_state = cir.run(state)
        # Measure the circuit and record the measurement results 
        after_state = self.measure(status=after_state, which_qubits=('Bob', 0), results_desired=res)
        return after_state
    
    def forward(self):
        # Training steps
        # Quantum state after Alice's operation
        phi = self.A_circuit(theta=self.theta1, state=self.phi, res=['0', '1'])
        psi = self.A_circuit(theta=self.theta1, state=self.psi, res=['0', '1'])

        # Calculate the loss function
        loss = 0
        for each_phi in phi:
            if each_phi.measured_result == '0':
                # maybe we should call it phi_1 which means predicting the phi to psi
                phi_1 = self.B_circuit(self.theta2, state=each_phi, res='1')
                loss += phi_1.prob
            elif each_phi.measured_result == '1':
                psi_1 = self.B_circuit(self.theta3, state=each_phi, res='1')
                loss += psi_1.prob

        for each_psi in psi:
            if each_psi.measured_result == '0':
                phi_0 = self.B_circuit(self.theta2, state=each_psi, res='0')
                loss += phi_0.prob
            elif each_psi.measured_result == '1':
                psi_0 = self.B_circuit(self.theta3, state=each_psi, res='0')
                loss += psi_0.prob

        return loss

    def evaluate(self):
        # Test step
        choice = np.random.choice(['phi', 'psi'])
        if choice == 'phi':
            self.status = self.phi
        else:
            self.status = self.psi
        print('Charlie chooses the state', choice)

        # Alice's operations
        status = self.A_circuit(theta=self.theta1, state=self.status, res=['0', '1'])
        # Bob's operations 
        result_0 = list()
        result_1 = list()
        for each_status in status:
            if each_status.measured_result == '0':
                phi = self.B_circuit(theta=self.theta2, state=each_status, res=['0', '1'])
                result_0.append(phi[0].prob.numpy())
                result_0.append(phi[1].prob.numpy())

            elif each_status.measured_result == '1':
                psi = self.B_circuit(theta=self.theta3, state=each_status, res=['0', '1'])
                result_1.append(psi[0].prob.numpy())
                result_1.append(psi[1].prob.numpy())
        print("The probability that Alice and Bob recognize it as phi:", result_0[0][0] + result_1[0][0])
        print("The probability that Alice and Bob recognize it as psi:", result_0[1][0] + result_1[1][0])
        

Train the QNN parameters, and Charlie randomly select one of the two orthogonal states $\lvert\psi\rangle$ and $\lvert\phi\rangle$ and see whether Alice and Bob can distinguish it correctly.

In [4]:
ITR = 100  # Set the number of training iterations
LR = 0.1   # Set learning rate
SEED = 999 # Fix randome seed for parameters in PQC
np.random.seed(999)


# Initialize dynamic graph mode
with fluid.dygraph.guard():
    net = Net()
    opt = fluid.optimizer.AdamOptimizer(learning_rate=LR, parameter_list=net.parameters())
    loss_list = list()
    # Train the LOCC net for ITR iterations by gradient descent
    for itr in range(ITR):
        loss = net()
        loss.backward()
        opt.minimize(loss)
        net.clear_gradients()
        loss_list.append(loss.numpy()[0])
        if itr % 10 == 0:
            print("itr " + str(itr) + ":", loss.numpy()[0])
    print("Minimum loss:", loss_list[-1])
    
    print("======================== test stage ===============================")
    np.random.seed(10)
    net.evaluate()
    np.random.seed(6)
    net.evaluate()

itr 0: 0.41451205406020003
itr 10: 0.028321852355736393
itr 20: 0.03680755589726499
itr 30: 0.020398244027092
itr 40: 0.00819374481825746
itr 50: 0.00456637111735521
itr 60: 0.00241819672311186
itr 70: 0.0010470019789628306
itr 80: 0.0004067586873393533
itr 90: 0.0001263131225914237
Minimum loss: 3.33357875113083e-05
======================== test stage ===============================
Charlie chooses the state psi
The probability that Alice and Bob recognize it as phi: 2.1562924160375818e-05
The probability that Alice and Bob recognize it as psi: 0.9999784370758396
Charlie chooses the state phi
The probability that Alice and Bob recognize it as phi: 0.9999920892819907
The probability that Alice and Bob recognize it as psi: 7.91071800851656e-06


## Conclusion

It can be seen from the simulation results that the trained quantum circuit can distinguish two orthogonal quantum states almost perfectly with an accuracy $>99.9\%$. There is an interesting question that can we generalize this discrimination scheme by adding more states to the category.

---
## References

[1] Barnett, Stephen M., and Sarah Croke. "Quantum state discrimination." [Advances in Optics and Photonics 1.2 (2009): 238-278.](https://www.osapublishing.org/abstract.cfm?id=176580)

[2] Chefles, Anthony. "Quantum state discrimination." [Contemporary Physics 41.6 (2000): 401-424.](https://arxiv.org/abs/quant-ph/0010114)

[3] Walgate, Jonathan, et al. "Local distinguishability of multipartite orthogonal quantum states." [Physical Review Letters 85.23 (2000): 4972.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.85.4972)
